In [50]:
import os
import pandas as pd
import glob
import pandas as pd
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
import re
from const import QUERY, CONNECT


In [51]:
# Define the input folder
input_folder = "./input/assign_input/"

# Get a list of files in the folder (xlsx or csv)
files = glob.glob(os.path.join(input_folder, "*.xlsx")) + glob.glob(os.path.join(input_folder, "*.csv"))
print(files)
# Read only the first file found
if files:
    file_path = files[0]
    # print(f"Reading file: {file_path}")
    
    if file_path.endswith(".xlsx"):
        assign = pd.read_excel(file_path,sheet_name='Data', dtype={'MobileNo': str})
    elif file_path.endswith(".csv"):
        assign = pd.read_csv(file_path,sheet_name='Data', dtype={'MobileNo': str})
    
    # print(assign.head())  # Display first few rows
else:
    print("No file found in the folder.")

['./input/assign_input\\SMS.xlsx', './input/assign_input\\~$SMS.xlsx']


In [52]:
def clean_column_names(df):
    df.columns = df.columns.str.lower()  
    df.columns = df.columns.str.replace(r'\.', '', regex=True)
    df.columns = df.columns.str.replace(r' ', '_', regex=True)  
    df.columns = df.columns.str.replace('customer_id_no|customer_no|customer_id|national_id', 'customer_no', regex=True) 
    df.columns = df.columns.str.replace(r'loan_no', 'contract_no', regex=True)
    # df.columns = df.columns.str.replace(r'mobile.*', 'mobile_no', regex=True)
    df.columns = df.columns.str.replace(r'customer_name/surname\(thai\)', 'customer_name', regex=True)  
    return df

In [53]:
assign_delay = clean_column_names(assign)

In [54]:
assign_delay

,contract_no,loan_balance,overduedays,overduecnt,product_code,mobileno,flag,4_digit,len,mobile_phone,bucket
0,8002059101031431,498507.80,0,99:NORMAL,A301:RL,0815545339,Y,1431,10,NaN,NaN
1,8002059101033361,449129.03,0,99:NORMAL,A301:RL,0627826235,NEW,3361,10,NaN,NaN
2,8002059101035011,359999.66,0,99:NORMAL,A301:RL,0812095739,Y,5011,10,NaN,NaN
3,8002059091065974,350190.59,0,99:NORMAL,A301:RL,0955246453,Y,5974,10,NaN,NaN
4,8002059091068906,333053.92,0,99:NORMAL,A301:RL,0924155363,Y,8906,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11284,9010221082000030,3300.91,0,99:NORMAL,A501:TL,0911123440,NEW,30,10,NaN,NaN
11285,9000220082000058,2952.45,0,99:NORMAL,A501:TL,0818522951,NEW,58,10,NaN,NaN
11286,9000221092012556,2060.88,0,99:NORMAL,A501:TL,0830711350,NEW,2556,10,NaN,NaN
11287,7240450102555445,1920.24,0,99:NORMAL,A501:TL,0927377224,Y,5445,10,NaN,NaN


In [55]:
customer = pd.read_csv('./input/customer_input/customerdata.csv', dtype={'MOBILE_PHONE_NO_VAL': str, 
                                                            'NATIONAL_ID': str, 
                                                            'CONTRACT_NO_VAL': str})

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_51560\245044610.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  customer = pd.read_csv('./input/customer_input/customerdata.csv', dtype={'MOBILE_PHONE_NO_VAL': str,


In [56]:
customer

,AS_OF_DATE,CONTRACT_NO_VAL,customer_no,CHANNEL,MONTHLY_INST_AMT,FIRST_DUE_DATE,NAME_VAL,SURNAME_VAL,MOBILE_PHONE_NO_VAL,RESIDENT_ADDRESS
0,2025-03-25 00:00:00.000,7240270102168513,1100501209015,NaN,0.0,2024-04-01 00:00:00.000,อารียา,บัวชะตา,0917240560,18/138 หมู่ที่ 6 ต. คลองสาม อ. คลองหลวง จ. ป...
1,2025-03-25 00:00:00.000,7231270101802967,3310101022303,NaN,0.0,2024-02-01 00:00:00.000,ประมวล,ทามูล,0801515085,183 หมู่ที่ 10 ต. บ้านบัว อ. เมืองบุรีรัมย์ ...
2,2025-03-25 00:00:00.000,7240140102019842,1801500073998,NaN,1521.0,2024-03-01 00:00:00.000,กมลรัตน์,แก้วสังข์,0992357369,501/5 บ้านเลขที่ 213/4 ซอยสาธุประดิษฐ์ 34 แยก6...
3,2025-03-25 00:00:00.000,7231270101803090,1330900124291,NaN,0.0,2024-02-01 00:00:00.000,พรพิไล,อุดทา,0801809886,67 หมู่ที่ 1 ต. หนองบัวดง อ. ศิลาลาด จ. ศรีส...
4,2025-03-25 00:00:00.000,7240130102013355,1860700151622,NaN,0.0,2024-03-01 00:00:00.000,แพงศรี,สินธุกูฎ,0816199064,เลขที่ 130/12 หมู่ 2 ต. บึง อ. ศรีราชา จ. ชล...
...,...,...,...,...,...,...,...,...,...,...
5425333,2025-03-25 00:00:00.000,7221170100026977,1101500985284,NaN,0.0,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5425334,2025-03-25 00:00:00.000,7230570100476339,1101500985284,NaN,0.0,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN
5425335,2025-03-25 00:00:00.000,7230670100644961,1101500985284,NaN,0.0,2023-07-01 00:00:00.000,NaN,NaN,NaN,NaN
5425336,2025-03-25 00:00:00.000,7250370105027976,3251200829045,NaN,0.0,2025-05-01 00:00:00.000,NaN,NaN,NaN,NaN


In [57]:
conn = jaydebeapi.connect(
        CONNECT.DB,
        CONNECT.PORT,
        CONNECT.USER,
        CONNECT.CNN,
        )
cur = conn.cursor()  

sms_type = pd.read_sql(f'''
    SELECT * FROM SIREETRON.REF_SMS_WORDING 
    ''', conn) 
conn.close()

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_51560\3062639091.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sms_type = pd.read_sql(f'''


In [58]:
sms_type

,ID,SMS_TYPE,SMS_WORDING
0,1,C4C,สินเชื่อรถยนต์แคชจอยของคุณลงท้าย {4digit} ครบก...
1,2,Unsave,สินเชื่อลงท้าย {4digit} ของท่านมียอดค้างชำระ โ...
2,3,In-House Unsave,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
3,4,SF+ Unsave (OD1-OD2),สินเชื่อของท่านค้างชำระ {totalunpaidbalance} บ...
4,5,SF+ non assign,สินเชื่อ Samsung บัญชีลงท้าย {4digit} เกินกำหน...
5,6,Delayed Calling,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
6,7,Offer TDR (old),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
7,8,TDR (GPD/Normal),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
8,9,TDR (SPD only),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
9,10,Offer Settlement,KBJ Capital ขอเสนอส่วนลดพิเศษสูงสุด 30% สำหรับ...


In [59]:
assign_delay

,contract_no,loan_balance,overduedays,overduecnt,product_code,mobileno,flag,4_digit,len,mobile_phone,bucket
0,8002059101031431,498507.80,0,99:NORMAL,A301:RL,0815545339,Y,1431,10,NaN,NaN
1,8002059101033361,449129.03,0,99:NORMAL,A301:RL,0627826235,NEW,3361,10,NaN,NaN
2,8002059101035011,359999.66,0,99:NORMAL,A301:RL,0812095739,Y,5011,10,NaN,NaN
3,8002059091065974,350190.59,0,99:NORMAL,A301:RL,0955246453,Y,5974,10,NaN,NaN
4,8002059091068906,333053.92,0,99:NORMAL,A301:RL,0924155363,Y,8906,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11284,9010221082000030,3300.91,0,99:NORMAL,A501:TL,0911123440,NEW,30,10,NaN,NaN
11285,9000220082000058,2952.45,0,99:NORMAL,A501:TL,0818522951,NEW,58,10,NaN,NaN
11286,9000221092012556,2060.88,0,99:NORMAL,A501:TL,0830711350,NEW,2556,10,NaN,NaN
11287,7240450102555445,1920.24,0,99:NORMAL,A501:TL,0927377224,Y,5445,10,NaN,NaN


In [60]:
customer

,AS_OF_DATE,CONTRACT_NO_VAL,customer_no,CHANNEL,MONTHLY_INST_AMT,FIRST_DUE_DATE,NAME_VAL,SURNAME_VAL,MOBILE_PHONE_NO_VAL,RESIDENT_ADDRESS
0,2025-03-25 00:00:00.000,7240270102168513,1100501209015,NaN,0.0,2024-04-01 00:00:00.000,อารียา,บัวชะตา,0917240560,18/138 หมู่ที่ 6 ต. คลองสาม อ. คลองหลวง จ. ป...
1,2025-03-25 00:00:00.000,7231270101802967,3310101022303,NaN,0.0,2024-02-01 00:00:00.000,ประมวล,ทามูล,0801515085,183 หมู่ที่ 10 ต. บ้านบัว อ. เมืองบุรีรัมย์ ...
2,2025-03-25 00:00:00.000,7240140102019842,1801500073998,NaN,1521.0,2024-03-01 00:00:00.000,กมลรัตน์,แก้วสังข์,0992357369,501/5 บ้านเลขที่ 213/4 ซอยสาธุประดิษฐ์ 34 แยก6...
3,2025-03-25 00:00:00.000,7231270101803090,1330900124291,NaN,0.0,2024-02-01 00:00:00.000,พรพิไล,อุดทา,0801809886,67 หมู่ที่ 1 ต. หนองบัวดง อ. ศิลาลาด จ. ศรีส...
4,2025-03-25 00:00:00.000,7240130102013355,1860700151622,NaN,0.0,2024-03-01 00:00:00.000,แพงศรี,สินธุกูฎ,0816199064,เลขที่ 130/12 หมู่ 2 ต. บึง อ. ศรีราชา จ. ชล...
...,...,...,...,...,...,...,...,...,...,...
5425333,2025-03-25 00:00:00.000,7221170100026977,1101500985284,NaN,0.0,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5425334,2025-03-25 00:00:00.000,7230570100476339,1101500985284,NaN,0.0,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN
5425335,2025-03-25 00:00:00.000,7230670100644961,1101500985284,NaN,0.0,2023-07-01 00:00:00.000,NaN,NaN,NaN,NaN
5425336,2025-03-25 00:00:00.000,7250370105027976,3251200829045,NaN,0.0,2025-05-01 00:00:00.000,NaN,NaN,NaN,NaN


In [29]:
customer.columns = customer.columns.str.lower()  
customer

,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-03-25 00:00:00.000,7240270102168513,1100501209015,NaN,0.0,2024-04-01 00:00:00.000,อารียา,บัวชะตา,0917240560,18/138 หมู่ที่ 6 ต. คลองสาม อ. คลองหลวง จ. ป...
1,2025-03-25 00:00:00.000,7231270101802967,3310101022303,NaN,0.0,2024-02-01 00:00:00.000,ประมวล,ทามูล,0801515085,183 หมู่ที่ 10 ต. บ้านบัว อ. เมืองบุรีรัมย์ ...
2,2025-03-25 00:00:00.000,7240140102019842,1801500073998,NaN,1521.0,2024-03-01 00:00:00.000,กมลรัตน์,แก้วสังข์,0992357369,501/5 บ้านเลขที่ 213/4 ซอยสาธุประดิษฐ์ 34 แยก6...
3,2025-03-25 00:00:00.000,7231270101803090,1330900124291,NaN,0.0,2024-02-01 00:00:00.000,พรพิไล,อุดทา,0801809886,67 หมู่ที่ 1 ต. หนองบัวดง อ. ศิลาลาด จ. ศรีส...
4,2025-03-25 00:00:00.000,7240130102013355,1860700151622,NaN,0.0,2024-03-01 00:00:00.000,แพงศรี,สินธุกูฎ,0816199064,เลขที่ 130/12 หมู่ 2 ต. บึง อ. ศรีราชา จ. ชล...
...,...,...,...,...,...,...,...,...,...,...
5425333,2025-03-25 00:00:00.000,7221170100026977,1101500985284,NaN,0.0,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5425334,2025-03-25 00:00:00.000,7230570100476339,1101500985284,NaN,0.0,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN
5425335,2025-03-25 00:00:00.000,7230670100644961,1101500985284,NaN,0.0,2023-07-01 00:00:00.000,NaN,NaN,NaN,NaN
5425336,2025-03-25 00:00:00.000,7250370105027976,3251200829045,NaN,0.0,2025-05-01 00:00:00.000,NaN,NaN,NaN,NaN


In [61]:
assign_delay

,contract_no,loan_balance,overduedays,overduecnt,product_code,mobileno,flag,4_digit,len,mobile_phone,bucket
0,8002059101031431,498507.80,0,99:NORMAL,A301:RL,0815545339,Y,1431,10,NaN,NaN
1,8002059101033361,449129.03,0,99:NORMAL,A301:RL,0627826235,NEW,3361,10,NaN,NaN
2,8002059101035011,359999.66,0,99:NORMAL,A301:RL,0812095739,Y,5011,10,NaN,NaN
3,8002059091065974,350190.59,0,99:NORMAL,A301:RL,0955246453,Y,5974,10,NaN,NaN
4,8002059091068906,333053.92,0,99:NORMAL,A301:RL,0924155363,Y,8906,10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11284,9010221082000030,3300.91,0,99:NORMAL,A501:TL,0911123440,NEW,30,10,NaN,NaN
11285,9000220082000058,2952.45,0,99:NORMAL,A501:TL,0818522951,NEW,58,10,NaN,NaN
11286,9000221092012556,2060.88,0,99:NORMAL,A501:TL,0830711350,NEW,2556,10,NaN,NaN
11287,7240450102555445,1920.24,0,99:NORMAL,A501:TL,0927377224,Y,5445,10,NaN,NaN


In [62]:
customer

,AS_OF_DATE,CONTRACT_NO_VAL,customer_no,CHANNEL,MONTHLY_INST_AMT,FIRST_DUE_DATE,NAME_VAL,SURNAME_VAL,MOBILE_PHONE_NO_VAL,RESIDENT_ADDRESS
0,2025-03-25 00:00:00.000,7240270102168513,1100501209015,NaN,0.0,2024-04-01 00:00:00.000,อารียา,บัวชะตา,0917240560,18/138 หมู่ที่ 6 ต. คลองสาม อ. คลองหลวง จ. ป...
1,2025-03-25 00:00:00.000,7231270101802967,3310101022303,NaN,0.0,2024-02-01 00:00:00.000,ประมวล,ทามูล,0801515085,183 หมู่ที่ 10 ต. บ้านบัว อ. เมืองบุรีรัมย์ ...
2,2025-03-25 00:00:00.000,7240140102019842,1801500073998,NaN,1521.0,2024-03-01 00:00:00.000,กมลรัตน์,แก้วสังข์,0992357369,501/5 บ้านเลขที่ 213/4 ซอยสาธุประดิษฐ์ 34 แยก6...
3,2025-03-25 00:00:00.000,7231270101803090,1330900124291,NaN,0.0,2024-02-01 00:00:00.000,พรพิไล,อุดทา,0801809886,67 หมู่ที่ 1 ต. หนองบัวดง อ. ศิลาลาด จ. ศรีส...
4,2025-03-25 00:00:00.000,7240130102013355,1860700151622,NaN,0.0,2024-03-01 00:00:00.000,แพงศรี,สินธุกูฎ,0816199064,เลขที่ 130/12 หมู่ 2 ต. บึง อ. ศรีราชา จ. ชล...
...,...,...,...,...,...,...,...,...,...,...
5425333,2025-03-25 00:00:00.000,7221170100026977,1101500985284,NaN,0.0,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5425334,2025-03-25 00:00:00.000,7230570100476339,1101500985284,NaN,0.0,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN
5425335,2025-03-25 00:00:00.000,7230670100644961,1101500985284,NaN,0.0,2023-07-01 00:00:00.000,NaN,NaN,NaN,NaN
5425336,2025-03-25 00:00:00.000,7250370105027976,3251200829045,NaN,0.0,2025-05-01 00:00:00.000,NaN,NaN,NaN,NaN


In [63]:
customer = clean_column_names(customer)

In [64]:
customer


,as_of_date,contract_no_val,customer_no,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-03-25 00:00:00.000,7240270102168513,1100501209015,NaN,0.0,2024-04-01 00:00:00.000,อารียา,บัวชะตา,0917240560,18/138 หมู่ที่ 6 ต. คลองสาม อ. คลองหลวง จ. ป...
1,2025-03-25 00:00:00.000,7231270101802967,3310101022303,NaN,0.0,2024-02-01 00:00:00.000,ประมวล,ทามูล,0801515085,183 หมู่ที่ 10 ต. บ้านบัว อ. เมืองบุรีรัมย์ ...
2,2025-03-25 00:00:00.000,7240140102019842,1801500073998,NaN,1521.0,2024-03-01 00:00:00.000,กมลรัตน์,แก้วสังข์,0992357369,501/5 บ้านเลขที่ 213/4 ซอยสาธุประดิษฐ์ 34 แยก6...
3,2025-03-25 00:00:00.000,7231270101803090,1330900124291,NaN,0.0,2024-02-01 00:00:00.000,พรพิไล,อุดทา,0801809886,67 หมู่ที่ 1 ต. หนองบัวดง อ. ศิลาลาด จ. ศรีส...
4,2025-03-25 00:00:00.000,7240130102013355,1860700151622,NaN,0.0,2024-03-01 00:00:00.000,แพงศรี,สินธุกูฎ,0816199064,เลขที่ 130/12 หมู่ 2 ต. บึง อ. ศรีราชา จ. ชล...
...,...,...,...,...,...,...,...,...,...,...
5425333,2025-03-25 00:00:00.000,7221170100026977,1101500985284,NaN,0.0,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5425334,2025-03-25 00:00:00.000,7230570100476339,1101500985284,NaN,0.0,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN
5425335,2025-03-25 00:00:00.000,7230670100644961,1101500985284,NaN,0.0,2023-07-01 00:00:00.000,NaN,NaN,NaN,NaN
5425336,2025-03-25 00:00:00.000,7250370105027976,3251200829045,NaN,0.0,2025-05-01 00:00:00.000,NaN,NaN,NaN,NaN


In [65]:
# assign_delay = assign_delay.merge(sms_type,left_on='sms_type',right_on = 'sms_type', how='left')
# assign_delay['customer_no'] = assign_delay['customer_no'].astype(str)
assign_delay['contract_no'] = assign_delay['contract_no'].astype(str)
customer['contract_no_val'] = customer['contract_no_val'].astype(str)
# customer['national_id'] = customer['national_id'].astype(str) 

In [67]:
# assign_delay_merge = assign_delay.merge(customer, left_on=['customer_no', 'contract_no'], right_on=['national_id' ,'contract_no_val'], how='left')
assign_delay_merge = assign_delay.merge(customer, left_on= 'contract_no', right_on='contract_no_val', how='left')


In [68]:
assign_delay_merge.columns

Index(['contract_no', 'loan_balance', 'overduedays', 'overduecnt',
       'product_code', 'mobileno', 'flag', '4_digit', 'len', 'mobile_phone',
       'bucket', 'as_of_date', 'contract_no_val', 'customer_no', 'channel',
       'monthly_inst_amt', 'first_due_date', 'name_val', 'surname_val',
       'mobile_phone_no_val', 'resident_address'],
      dtype='object')

In [70]:
assign_delay_merge['mobile_check'] = assign_delay_merge.apply(
    lambda row: True if row['mobileno'] == row['mobile_phone_no_val'] else row['mobile_phone_no_val'],
    axis=1
)

In [71]:
assign_delay_merge['last4digit_val'] = assign_delay_merge['contract_no'].str[-4:]
# assign_delay_merge['sms_wording'] = assign_delay_merge.apply(lambda row: row['sms_wording'].replace("{4digit}", str(row['last4digit_val'])), axis=1)


In [72]:
assign_delay_merge.to_excel('./output/iiii.xlsx')